In [15]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_selection import RFE, RFECV


# Reading dataset
train = pd.read_csv('train_dataset_1.csv')
train['orderDate'] = pd.to_datetime(train['orderDate'], format = '%Y-%m-%d')
train['deliveryDate'] = pd.to_datetime(train['deliveryDate'], format = '%Y-%m-%d')
train['dateOfBirth'] = pd.to_datetime(train['dateOfBirth'], format = "%Y-%m-%d")
train['creationDate'] = pd.to_datetime(train['creationDate'], format = "%Y-%m-%d")
train.head()

,orderItemID,orderDate,deliveryDate,itemID,size,color,manufacturerID,price,customerID,salutation,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,interaction_1,interaction_2,interaction_3,interaction_4
0,1,2012-04-01,2012-04-03,186,m,denim,25,69.90,794,Mrs,...,0,0,0,0,0,0,0,0,1,0
1,2,2012-04-01,2012-04-03,71,9+,ocher,21,69.95,794,Mrs,...,0,0,0,0,0,0,0,0,1,0
2,3,2012-04-01,2012-04-03,71,9+,curry,21,69.95,794,Mrs,...,0,0,0,0,0,0,0,0,1,0
3,4,2012-04-02,NaT,22,m,green,14,39.90,808,Mrs,...,0,0,0,0,0,0,1,0,0,0
4,5,2012-04-02,1990-12-31,151,39,black,53,29.90,825,Mrs,...,1,0,0,0,0,0,0,0,0,0


In [2]:
train.columns

Index(['orderItemID', 'orderDate', 'deliveryDate', 'itemID', 'size', 'color',
       'manufacturerID', 'price', 'customerID', 'salutation', 'dateOfBirth',
       'state', 'creationDate', 'returnShipment', 'days_to_delivery',
       'Customer_age', 'manufacturer_highest_return_rate',
       'item_highest_return_rate', 'feature_100', 'size_highest_return_rate',
       'clothes', 'state_highest_return_rate',
       'salutation_highest_return_rate', 'Item_Average_Price',
       'price_higher_average', 'Company', 'Family', 'Mr', 'Mrs',
       'not reported', 'salutation_numb', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'interaction_1', 'interaction_2', 'interaction_3',
       'interaction_4'],
      dtype='object')

In [16]:
import Feature_Importance_Funs

X = train.drop(['returnShipment', 'orderItemID', 'orderDate', 'deliveryDate', 'itemID', 'color', 'manufacturerID', 
               'customerID', 'salutation', 'dateOfBirth', 'state', 'creationDate', 'size'], axis = 1)
Y = train['returnShipment']

X = X.dropna()
Y = Y.dropna()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

X.head()

,price,days_to_delivery,Customer_age,manufacturer_highest_return_rate,item_highest_return_rate,feature_100,size_highest_return_rate,clothes,state_highest_return_rate,salutation_highest_return_rate,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,interaction_1,interaction_2,interaction_3,interaction_4
0,69.90,2.0,47.0,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,69.95,2.0,47.0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,69.95,2.0,47.0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,39.90,11.0,52.0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,1,0,0,0
4,29.90,11.0,48.0,1,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
import Feature_Importance_Funs

nsim = 10
cv = 5
test = Feature_Importance_Funs.RF_Feature_Importance(X, Y, nsim, cv)

In [18]:
feature_importance = pd.DataFrame({'Features': X.columns, 'Importance': test.apply(np.mean, axis = 0)})
feature_importance['Importance'] = round(feature_importance['Importance'], 4) * 100
feature_importance = feature_importance.sort_values(by = ['Importance'], ascending = False).reset_index(drop = True)
feature_importance.head()

,Features,Importance
0,interaction_1,24.06
1,interaction_3,17.89
2,item_highest_return_rate,15.95
3,days_to_delivery,9.99
4,price,8.62


In [19]:
feature_importance

,Features,Importance
0,interaction_1,24.06
1,interaction_3,17.89
2,item_highest_return_rate,15.95
3,days_to_delivery,9.99
4,price,8.62
5,Item_Average_Price,7.35
6,feature_100,4.72
7,interaction_4,3.80
8,interaction_2,2.99
9,manufacturer_highest_return_rate,2.60


In [ ]:
#feature_importance.to_csv('feature_importances.csv', header = True, index = False, float_format='%g')

In [11]:
from sklearn.feature_selection import RFE, RFECV

# Running RFE with random forest
auto_feature_selection = RFECV(estimator = RandomForestClassifier(n_estimators = 100, max_depth = 5, min_samples_split = 10, min_samples_leaf = 7), step = 1, min_features_to_select = 2, cv = 5).fit(X_train, Y_train)

# Extracting features that got slected
print(X_train.columns[auto_feature_selection.support_])

Index(['price', 'days_to_delivery', 'item_highest_return_rate',
       'interaction_1', 'interaction_3'],
      dtype='object')
